---
## 1. Teoretický Úvod

### 1.1 Problém Neúplnosti Dat

**Situace:**
- OHLCV data: 10 let (2015-2025) ✅
- Fundamentální data: pouze ~1.5 roku (2024-2025) ⚠️

**Řešení:** Použít natrénovaný RF model pro **zpětnou imputaci** fundamentů.

### 1.2 Validita Přístupu

Proč je tento přístup validní?

| Argument | Vysvětlení |
|----------|------------|
| **Korelace OHLCV-Fundamenty** | Cenové vzory reflektují fundamentální hodnotu |
| **Konzistence v čase** | Fundamenty se mění pomalu (čtvrtletně) |
| **Cross-validation** | Model testován na out-of-sample datech |

### 1.3 Omezení

⚠️ **Důležité:**
- Predikované hodnoty jsou **aproximace**, ne přesná čísla
- Vhodné pro **trendovou analýzu**, ne pro přesné valuace
- Extrémní události (COVID, finanční krize) mohou být špatně zachyceny

---
## 2. Setup Prostředí

In [ ]:
# Instalace (pro Colab)
!pip install pandas numpy scikit-learn joblib matplotlib seaborn tqdm -q

print("✓ Knihovny nainstalovány")

In [ ]:
# Import knihoven
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from tqdm.notebook import tqdm
import warnings
import os
import joblib
import json

warnings.filterwarnings('ignore')
np.random.seed(42)

print("✓ Knihovny načteny")

In [ ]:
# Připojení Google Drive
try:
    from google.colab import drive
    drive.mount('/content/drive')
    DRIVE_PATH = '/content/drive/MyDrive/MachineLearning'
    RUNNING_ON_COLAB = True
    print(f"✓ Google Drive připojen: {DRIVE_PATH}")
except:
    DRIVE_PATH = '.'
    RUNNING_ON_COLAB = False
    print("ℹ️ Lokální prostředí")

# Cesty
DATA_PATH = f"{DRIVE_PATH}/data"
MODEL_PATH = f"{DRIVE_PATH}/models"
COMPLETE_PATH = f"{DATA_PATH}/complete"
os.makedirs(COMPLETE_PATH, exist_ok=True)

---
## 3. Načtení Dat a Modelu

In [ ]:
# Načtení natrénovaného modelu
model_path = f"{MODEL_PATH}/fundamental_predictor.pkl"
scaler_path = f"{MODEL_PATH}/feature_scaler.pkl"
metadata_path = f"{MODEL_PATH}/fundamental_predictor_metadata.json"

model = joblib.load(model_path)
scaler = joblib.load(scaler_path)

with open(metadata_path, 'r') as f:
    metadata = json.load(f)

FEATURE_COLS = metadata['features']
TARGET_COLS = metadata['targets']

print(f"✓ Model načten")
print(f"   Features: {len(FEATURE_COLS)}")
print(f"   Targets: {len(TARGET_COLS)}")
print(f"   Průměrné R²: {metadata['avg_r2']:.3f}")

In [ ]:
# Načtení OHLCV dat
ohlcv_path = f"{DATA_PATH}/ohlcv/all_sectors_ohlcv_10y.csv"
ohlcv_df = pd.read_csv(ohlcv_path, parse_dates=['date'])

print(f"\n📈 OHLCV Data:")
print(f"   Záznamů: {len(ohlcv_df):,}")
print(f"   Tickerů: {ohlcv_df['ticker'].nunique()}")
print(f"   Období: {ohlcv_df['date'].min().strftime('%Y-%m')} → {ohlcv_df['date'].max().strftime('%Y-%m')}")

In [ ]:
# Načtení reálných fundamentů
fund_path = f"{DATA_PATH}/fundamentals/all_sectors_fundamentals.csv"
real_fundamentals = pd.read_csv(fund_path, parse_dates=['date'])

print(f"\n📊 Reálné Fundamenty:")
print(f"   Záznamů: {len(real_fundamentals)}")

---
## 4. Imputace Historických Fundamentů

### 4.1 Strategie

Pro každý měsíc v období 2015-2024:
1. Extrahuj OHLCV features
2. Standardizuj features
3. Predikuj 14 fundamentálních metrik
4. Přidej k datasetu

In [ ]:
def impute_fundamentals(ohlcv: pd.DataFrame, model, scaler, 
                        feature_cols: list, target_cols: list) -> pd.DataFrame:
    """
    Imputuje fundamentální metriky pomocí natrénovaného RF modelu.
    
    Args:
        ohlcv: DataFrame s OHLCV daty a technickými indikátory
        model: Natrénovaný Multi-Output RF model
        scaler: StandardScaler pro features
        feature_cols: Seznam feature sloupců
        target_cols: Seznam target sloupců
    
    Returns:
        DataFrame s imputovanými fundamenty
    """
    result = ohlcv.copy()
    
    # Kontrola dostupných features
    available_features = [f for f in feature_cols if f in result.columns]
    
    if len(available_features) < len(feature_cols):
        missing = set(feature_cols) - set(available_features)
        print(f"⚠️ Chybějící features: {missing}")
        return result
    
    # Odstranění řádků s NaN ve features
    valid_mask = result[available_features].notna().all(axis=1)
    valid_data = result[valid_mask].copy()
    
    print(f"📊 Imputace pro {len(valid_data):,} záznamů...")
    
    if len(valid_data) == 0:
        print("⚠️ Žádná validní data pro imputaci")
        return result
    
    # Extrakce features
    X = valid_data[available_features].values
    
    # Standardizace
    X_scaled = scaler.transform(X)
    
    # Predikce
    predictions = model.predict(X_scaled)
    
    # Přidání predikcí do DataFrame
    for i, col in enumerate(target_cols):
        result.loc[valid_mask, col] = predictions[:, i]
    
    print(f"✓ Imputováno {len(target_cols)} metrik")
    
    return result

# Imputace
print("🔄 Imputace historických fundamentů...")
print("="*50)

imputed_df = impute_fundamentals(
    ohlcv_df, 
    model, 
    scaler, 
    FEATURE_COLS, 
    TARGET_COLS
)

In [ ]:
# Kontrola imputovaných hodnot
print("\n📊 STATISTIKY IMPUTOVANÝCH HODNOT")
print("="*60)

for col in TARGET_COLS:
    if col in imputed_df.columns:
        values = imputed_df[col].dropna()
        print(f"{col:<25} min={values.min():>10.2f}  max={values.max():>10.2f}  mean={values.mean():>10.2f}")

---
## 5. Validace Predikcí

### 5.1 Sanity Checks

Kontrolujeme, že predikované hodnoty jsou v rozumných rozmezích.

In [ ]:
def validate_predictions(df: pd.DataFrame, target_cols: list) -> dict:
    """
    Validuje predikované hodnoty pomocí sanity checks.
    
    Kontroly:
    - P/E by mělo být 0-100 (většina)
    - ROE/ROA by mělo být -1 až 1
    - Debt/Equity by mělo být >= 0
    """
    validation = {}
    
    # Definice očekávaných rozmezí
    expected_ranges = {
        'PE': (0, 200),
        'PB': (0, 50),
        'PS': (0, 100),
        'EV_EBITDA': (-10, 100),
        'ROE': (-2, 2),
        'ROA': (-1, 1),
        'Profit_Margin': (-1, 1),
        'Operating_Margin': (-1, 1),
        'Gross_Margin': (-1, 1),
        'Debt_to_Equity': (0, 1000),
        'Current_Ratio': (0, 20),
        'Quick_Ratio': (0, 20),
        'Revenue_Growth_YoY': (-1, 5),
        'Earnings_Growth_YoY': (-5, 10)
    }
    
    print("📊 VALIDACE PREDIKCÍ")
    print("="*70)
    print(f"{'Metrika':<25} {'V rozmezí':>12} {'Pod':>10} {'Nad':>10} {'Status':>10}")
    print("-"*70)
    
    for col in target_cols:
        if col not in df.columns or col not in expected_ranges:
            continue
            
        values = df[col].dropna()
        if len(values) == 0:
            continue
            
        min_val, max_val = expected_ranges[col]
        
        in_range = ((values >= min_val) & (values <= max_val)).sum()
        below = (values < min_val).sum()
        above = (values > max_val).sum()
        
        in_range_pct = in_range / len(values) * 100
        
        status = '✅' if in_range_pct > 90 else '⚠️' if in_range_pct > 70 else '❌'
        
        validation[col] = {
            'in_range_pct': in_range_pct,
            'below': below,
            'above': above,
            'status': status
        }
        
        print(f"{col:<25} {in_range_pct:>11.1f}% {below:>10} {above:>10} {status:>10}")
    
    return validation

# Validace
validation_results = validate_predictions(imputed_df, TARGET_COLS)

In [ ]:
# Clipping extrémních hodnot
def clip_extreme_values(df: pd.DataFrame, target_cols: list) -> pd.DataFrame:
    """
    Ořízne extrémní hodnoty na rozumné rozmezí.
    """
    result = df.copy()
    
    clip_ranges = {
        'PE': (0, 200),
        'PB': (0, 50),
        'PS': (0, 100),
        'EV_EBITDA': (-10, 100),
        'ROE': (-2, 2),
        'ROA': (-1, 1),
        'Profit_Margin': (-1, 1),
        'Operating_Margin': (-1, 1),
        'Gross_Margin': (0, 1),
        'Debt_to_Equity': (0, 500),
        'Current_Ratio': (0, 10),
        'Quick_Ratio': (0, 10),
        'Revenue_Growth_YoY': (-1, 3),
        'Earnings_Growth_YoY': (-3, 5)
    }
    
    for col in target_cols:
        if col in result.columns and col in clip_ranges:
            min_val, max_val = clip_ranges[col]
            before = result[col].isna().sum()
            result[col] = result[col].clip(min_val, max_val)
            
    print("✓ Extrémní hodnoty oříznuty")
    return result

# Clipping
imputed_df = clip_extreme_values(imputed_df, TARGET_COLS)

---
## 6. Merge s Reálnými Daty

Pro období 2024-2025 používáme **reálná data** místo predikcí.

In [ ]:
def merge_with_real_data(imputed: pd.DataFrame, real: pd.DataFrame, 
                         target_cols: list, cutoff_date: str = '2024-01-01') -> pd.DataFrame:
    """
    Nahradí imputované hodnoty reálnými daty pro období po cutoff_date.
    
    Args:
        imputed: DataFrame s imputovanými hodnotami
        real: DataFrame s reálnými fundamenty
        target_cols: Seznam target sloupců
        cutoff_date: Datum od kterého použít reálná data
    
    Returns:
        Merged DataFrame
    """
    result = imputed.copy()
    cutoff = pd.to_datetime(cutoff_date)
    
    # Pro každý ticker nahradíme imputované hodnoty reálnými
    replaced_count = 0
    
    for ticker in real['ticker'].unique():
        ticker_real = real[real['ticker'] == ticker]
        
        if ticker_real.empty:
            continue
        
        # Najdi záznamy po cutoff date
        mask = (result['ticker'] == ticker) & (result['date'] >= cutoff)
        
        # Nahraď imputované hodnoty reálnými
        for col in target_cols:
            if col in ticker_real.columns and col in result.columns:
                real_value = ticker_real[col].values[0]
                if pd.notna(real_value):
                    result.loc[mask, col] = real_value
                    replaced_count += mask.sum()
    
    print(f"✓ Nahrazeno {replaced_count:,} imputovaných hodnot reálnými daty")
    return result

# Merge
print("\n🔄 Merge s reálnými daty...")
complete_df = merge_with_real_data(imputed_df, real_fundamentals, TARGET_COLS)

---
## 7. Finalizace Datasetu

In [ ]:
# Přidání příznaku zdroje dat
cutoff = pd.to_datetime('2024-01-01')
complete_df['data_source'] = np.where(
    complete_df['date'] >= cutoff, 
    'real', 
    'imputed'
)

# Statistiky
source_counts = complete_df['data_source'].value_counts()
print(f"\n📊 Zdroj dat:")
print(f"   Imputované: {source_counts.get('imputed', 0):,}")
print(f"   Reálné: {source_counts.get('real', 0):,}")

In [ ]:
# Uložení kompletního datasetu
complete_path = f"{COMPLETE_PATH}/all_sectors_complete_10y.csv"
complete_df.to_csv(complete_path, index=False)

print(f"💾 Kompletní dataset uložen: {complete_path}")
print(f"   Záznamů: {len(complete_df):,}")
print(f"   Sloupců: {len(complete_df.columns)}")

In [ ]:
# Uložení po sektorech
for sector in complete_df['sector'].unique():
    sector_df = complete_df[complete_df['sector'] == sector]
    sector_path = f"{COMPLETE_PATH}/{sector}_complete_10y.csv"
    sector_df.to_csv(sector_path, index=False)
    print(f"💾 {sector}: {len(sector_df):,} záznamů → {sector_path}")

---
## 8. Vizualizace

In [ ]:
# Vizualizace imputovaných vs reálných dat
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. P/E v čase pro jeden ticker
ax1 = axes[0, 0]
sample_ticker = complete_df['ticker'].unique()[0]
sample_data = complete_df[complete_df['ticker'] == sample_ticker].sort_values('date')

imputed_mask = sample_data['data_source'] == 'imputed'
real_mask = sample_data['data_source'] == 'real'

ax1.plot(sample_data[imputed_mask]['date'], sample_data[imputed_mask]['PE'], 
         'b-', label='Imputované', alpha=0.7)
ax1.plot(sample_data[real_mask]['date'], sample_data[real_mask]['PE'], 
         'r-', label='Reálné', linewidth=2)
ax1.axvline(pd.to_datetime('2024-01-01'), color='green', linestyle='--', 
            label='Cutoff (2024)')
ax1.set_title(f'P/E Ratio v Čase: {sample_ticker}', fontweight='bold')
ax1.set_xlabel('Datum')
ax1.set_ylabel('P/E')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Distribuce imputovaných vs reálných
ax2 = axes[0, 1]
imputed_pe = complete_df[complete_df['data_source'] == 'imputed']['PE'].dropna()
real_pe = complete_df[complete_df['data_source'] == 'real']['PE'].dropna()

ax2.hist(imputed_pe, bins=30, alpha=0.5, label='Imputované', color='blue', density=True)
ax2.hist(real_pe, bins=30, alpha=0.5, label='Reálné', color='red', density=True)
ax2.set_title('Distribuce P/E: Imputované vs Reálné', fontweight='bold')
ax2.set_xlabel('P/E Ratio')
ax2.set_ylabel('Hustota')
ax2.legend()

# 3. Počet záznamů v čase
ax3 = axes[1, 0]
monthly_counts = complete_df.groupby([complete_df['date'].dt.to_period('Y'), 'data_source']).size().unstack(fill_value=0)
monthly_counts.plot(kind='bar', stacked=True, ax=ax3, color=['blue', 'red'])
ax3.set_title('Počet Záznamů po Letech', fontweight='bold')
ax3.set_xlabel('Rok')
ax3.set_ylabel('Počet záznamů')
ax3.tick_params(axis='x', rotation=45)
ax3.legend(['Imputované', 'Reálné'])

# 4. Korelace fundamentů
ax4 = axes[1, 1]
fund_cols = ['PE', 'PB', 'ROE', 'ROA', 'Debt_to_Equity']
available_fund = [c for c in fund_cols if c in complete_df.columns]
corr = complete_df[available_fund].corr()
sns.heatmap(corr, annot=True, cmap='RdYlGn', center=0, ax=ax4, fmt='.2f')
ax4.set_title('Korelace Fundamentálních Metrik', fontweight='bold')

plt.tight_layout()
plt.savefig(f"{DATA_PATH}/data_completion_overview.png", dpi=150, bbox_inches='tight')
plt.show()

print(f"\n💾 Graf uložen: {DATA_PATH}/data_completion_overview.png")

---
## 9. Shrnutí

### ✅ Dokončeno:

| Úkol | Status |
|------|--------|
| Načtení modelu z Notebooku 02 | ✅ |
| Imputace 2015-2024 | ✅ |
| Validace predikcí | ✅ |
| Merge s reálnými daty | ✅ |
| Uložení kompletního datasetu | ✅ |

### 📁 Vytvořené soubory:

| Soubor | Popis |
|--------|-------|
| `data/complete/all_sectors_complete_10y.csv` | Kompletní 10-letý dataset |
| `data/complete/{Sector}_complete_10y.csv` | Sektorové datasety |

### ➡️ Další notebook:

**Notebook 04: Trénování Price Classifier**
- Ternární klasifikace: DOWN / HOLD / UP
- Random Forest Classifier
- Confusion Matrix, Precision, Recall, F1

In [ ]:
# Finální shrnutí
print("="*70)
print("📊 NOTEBOOK 03 - SHRNUTÍ")
print("="*70)

print(f"\n📈 Kompletní Dataset:")
print(f"   Záznamů: {len(complete_df):,}")
print(f"   Tickerů: {complete_df['ticker'].nunique()}")
print(f"   Sektorů: {complete_df['sector'].nunique()}")
print(f"   Období: {complete_df['date'].min().strftime('%Y-%m')} → {complete_df['date'].max().strftime('%Y-%m')}")

print(f"\n📊 Sloupce ({len(complete_df.columns)}):")
print(f"   OHLCV: 5")
print(f"   Technické: {len(FEATURE_COLS) - 5}")
print(f"   Fundamenty: {len(TARGET_COLS)}")

print(f"\n🔄 Zdroj dat:")
for source, count in source_counts.items():
    pct = count / len(complete_df) * 100
    print(f"   {source.capitalize()}: {count:,} ({pct:.1f}%)")

print(f"\n✅ Dataset připraven pro Notebook 04!")